In [ ]:
!pip install streamlit pyngrok timm peft


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 36.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 29.4 MB/s eta 0:00:00


In [ ]:
!pip install streamlit timm joblib opencv-python-headless torch torchvision peft -q
!npm install -g localtunnel -q

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸
added 22 packages in 3s
⠸
⠸3 packages are looking for funding
⠸  run `npm fund` for details
⠸

In [ ]:
# === DOWNLOAD MODEL DARI GITHUB ===

!wget -q https://raw.githubusercontent.com/Gammaa12/child-growth-classification/main/svm_stunting_model.pkl
!wget -q https://raw.githubusercontent.com/Gammaa12/child-growth-classification/main/scaler_stunting.pkl
!wget -q https://raw.githubusercontent.com/Gammaa12/child-growth-classification/main/cnn_ft_standard.pth
!wget -q https://raw.githubusercontent.com/Gammaa12/child-growth-classification/main/efficientnet_peft_lora_full.pth

print("✅ Semua model berhasil di-download")
!ls


✅ Semua model berhasil di-download
cnn_ft_standard.pth		 sample_data	      svm_stunting_model.pkl
efficientnet_peft_lora_full.pth  scaler_stunting.pkl


In [ ]:
from pyngrok import ngrok

ngrok.set_auth_token("36rugdt25HGeRAt6REpC8H0yhA9_2VRNkTKY5TWUNo5gkyGQc")


In [ ]:
# ============================
# PAGE CONFIG
# ============================
st.set_page_config(
    page_title="Child Growth Classification",
    page_icon="🧒",
    layout="wide"
)

# ============================
# CUSTOM CSS
# ============================
st.markdown("""
<style>
.main-title {
    font-size:42px;
    font-weight:800;
}
.sub-title {
    font-size:18px;
    color:#6c757d;
}
.card {
    background:#ffffff;
    padding:20px;
    border-radius:16px;
    box-shadow:0 8px 24px rgba(0,0,0,0.08);
}
.pred-label {
    font-size:20px;
    font-weight:700;
    text-align:center;
}
.conf {
    text-align:center;
    color:#555;
}
hr {
    margin: 1.5rem 0;
}
</style>
""", unsafe_allow_html=True)

# ============================
# HEADER
# ============================
st.markdown("<div class='main-title'>🧒 Child Growth Classification</div>", unsafe_allow_html=True)
st.markdown("<div class='sub-title'>Multi-Model Inference: SVM · CNN · EfficientNet + LoRA</div>", unsafe_allow_html=True)
st.write("")

# ============================
# SIDEBAR (CONTROL PANEL)
# ============================
with st.sidebar:
    st.markdown("## ⚙️ Control Panel")

    model_choice = st.selectbox(
        "🧠 Pilih Model",
        ["SVM (Classic)", "CNN Fine-Tuning", "EfficientNet + LoRA"]
    )

    uploaded_files = st.file_uploader(
        "📤 Upload Foto Wajah Anak",
        type=["jpg","png","jpeg"],
        accept_multiple_files=True
    )

    run_btn = st.button("🔍 Jalankan Prediksi", use_container_width=True)

    st.markdown("---")
    st.markdown("### ℹ️ Info Model")
    if model_choice == "SVM (Classic)":
        st.caption("✔ Grayscale\n✔ Fitur klasik\n✔ Cepat & ringan")
    elif model_choice == "CNN Fine-Tuning":
        st.caption("✔ CNN custom\n✔ Fine-tuned\n✔ End-to-end")
    else:
        st.caption("✔ EfficientNet-B0\n✔ LoRA Adaptation\n✔ Parameter Efficient")

# ============================
# MAIN CONTENT
# ============================
if uploaded_files and run_btn:

    images = []
    raw_images = []

    for file in uploaded_files:
        img = Image.open(file).convert("RGB")
        raw_images.append(img)
        images.append(cnn_tf(img))

    x = torch.stack(images)

    # ============================
    # PREVIEW
    # ============================
    st.markdown("## 🖼️ Preview Gambar")
    cols = st.columns(4)

    for i, (img, file) in enumerate(zip(raw_images, uploaded_files)):
        with cols[i % 4]:
            st.image(img, caption=file.name, use_container_width=True)

    st.markdown("<hr>", unsafe_allow_html=True)

    # ============================
    # INFERENCE
    # ============================
    if model_choice == "CNN Fine-Tuning":
        with torch.no_grad():
            out = cnn_model(x)
            probs = torch.softmax(out, 1).numpy()

    elif model_choice == "EfficientNet + LoRA":
        with torch.no_grad():
            out = eff_model(x)
            probs = torch.softmax(out, 1).numpy()

    else:  # SVM
        probs = []
        for img in raw_images:
            img_cv = cv2.resize(np.array(img), (64,64))
            gray = cv2.cvtColor(img_cv, cv2.COLOR_RGB2GRAY)
            feat = svm_scaler.transform(gray.flatten().reshape(1,-1))
            probs.append(svm_model.predict_proba(feat)[0])
        probs = np.array(probs)

    preds = probs.argmax(axis=1)

    # ============================
    # RESULT
    # ============================
    st.markdown("## 📊 Hasil Prediksi")
    cols = st.columns(4)

    for i, (img, pred, prob) in enumerate(zip(raw_images, preds, probs)):
        label = "VP-0 (Visually Proportional)" if pred == 0 else "VP-1 (Visually Linear)"
        conf = prob[pred] * 100

        with cols[i % 4]:
            st.markdown("<div class='card'>", unsafe_allow_html=True)
            st.image(img, use_container_width=True)
            st.markdown(f"<div class='pred-label'>{label}</div>", unsafe_allow_html=True)
            st.markdown(f"<div class='conf'>Confidence: {conf:.2f}%</div>", unsafe_allow_html=True)
            st.markdown("</div>", unsafe_allow_html=True)

else:
    st.info("⬅️ Silakan upload gambar dan jalankan prediksi melalui sidebar.")


2025-12-17 00:40:03.576 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-17 00:40:03.578 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-17 00:40:03.581 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-17 00:40:03.582 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-17 00:40:03.583 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-17 00:40:03.584 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-17 00:40:03.586 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-17 00:40:03.587 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [ ]:
!sed -n '1,40p' app.py


import streamlit as st
import torch
import torch.nn as nn
import timm
import joblib
import cv2
import numpy as np
from PIL import Image
import torchvision.transforms as T
from peft import LoraConfig, get_peft_model

# ============================
# PAGE CONFIG
# ============================
st.set_page_config(
    page_title="Child Growth Classification",
    page_icon="🧒",
    layout="centered"
)

st.markdown("""
<style>
.title { font-size:36px; font-weight:700; text-align:center; }
.subtitle { font-size:18px; text-align:center; color:gray; }
.card {
    background:#f9f9f9;
    padding:20px;
    border-radius:15px;
    box-shadow:0px 4px 10px rgba(0,0,0,0.1);
}
.result { font-size:22px; font-weight:700; text-align:center; }
</style>
""", unsafe_allow_html=True)

class ConvBlock(nn.Module):
    def __init__(self, in_ch, out_ch, pool=True):
        super().__init__()
        self.conv = nn.Conv2d(in_ch, out_ch, 3, padding=1, bias=False)
        self.bn = nn.BatchNorm2d(out_ch)
        s

In [ ]:
public_url = ngrok.connect(8501)
print(public_url)

!streamlit run app.py --server.port 8501


NgrokTunnel: "https://garbledly-cleanliest-stephani.ngrok-free.dev" -> "http://localhost:8501"





  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.125.244.30:8501

2025-12-17 00:56:51.789591: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-12-17 00:56:51.801079: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-12-17 00:56:51.829627: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765933011.872331   11960 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765933011.881523   11960 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when